In [17]:
!pip install transformers
!pip install torch

In [18]:
import pandas as pd

# Load your emails.csv
df = pd.read_csv("emails.csv")

# Inspect columns
df.head()
df.columns


Index(['file', 'message'], dtype='object')

In [23]:
import re
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

stop = set(stopwords.words('english'))

def clean_email(text):
    text = str(text).lower()
    text = re.sub(r'\n', ' ', text)
    text = re.sub(r'http\S+|www\S+', '', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = ' '.join(word for word in text.split() if word not in stop)
    return text

df['cleaned_text'] = df['message'].apply(clean_email)


df.head()


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\omson\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,file,message,cleaned_text
0,allen-p/_sent_mail/1.,Message-ID: <18782981.1075855378110.JavaMail.e...,message id javamail evans thyme date mon may p...
1,allen-p/_sent_mail/10.,Message-ID: <15464986.1075855378456.JavaMail.e...,message id javamail evans thyme date fri may p...
2,allen-p/_sent_mail/100.,Message-ID: <24216240.1075855687451.JavaMail.e...,message id javamail evans thyme date wed oct p...
3,allen-p/_sent_mail/1000.,Message-ID: <13505866.1075863688222.JavaMail.e...,message id javamail evans thyme date mon oct p...
4,allen-p/_sent_mail/1001.,Message-ID: <30922949.1075863688243.JavaMail.e...,message id javamail evans thyme date thu aug p...


In [24]:
from transformers import pipeline

# Load pre-trained DistilBERT sentiment model
sentiment_model = pipeline("sentiment-analysis")

# Example prediction
example_email = df['cleaned_text'][0]
result = sentiment_model(example_email)[0]
print(result)


No model was supplied, defaulted to distilbert/distilbert-base-uncased-finetuned-sst-2-english and revision 714eb0f (https://huggingface.co/distilbert/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.
Device set to use cpu


{'label': 'NEGATIVE', 'score': 0.9928846955299377}


In [25]:
import streamlit as st
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch
import numpy as np

st.title("📧 Email Sentiment Analyzer (with NEUTRAL Support)")

@st.cache_resource
def load_model():
    model_name = "cardiffnlp/twitter-roberta-base-sentiment-latest"
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    model = AutoModelForSequenceClassification.from_pretrained(model_name)
    return tokenizer, model

tokenizer, model = load_model()

def predict_sentiment(text):
    inputs = tokenizer(text, return_tensors="pt", truncation=True)
    outputs = model(**inputs)
    scores = outputs.logits.detach().numpy()[0]
    label_id = np.argmax(scores)
    labels = ["Negative", "Neutral", "Positive"]
    return labels[label_id], float(np.max(np.exp(scores) / np.sum(np.exp(scores))))

email_text = st.text_area("Enter email text:")

if st.button("Analyze Sentiment"):
    if email_text.strip() == "":
        st.warning("Please enter some email text!")
    else:
        sentiment, confidence = predict_sentiment(email_text)
        st.success(f"Sentiment: {sentiment} | Confidence: {confidence:.2f}")


2025-11-09 15:23:53.092 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-09 15:23:53.093 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-09 15:23:53.095 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-09 15:23:53.095 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-09 15:23:53.096 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-09 15:23:53.615 Thread 'Thread-19': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-09 15:23:53.617 Thread 'Thread-19': missing ScriptRunContext! This warning can be ignored when running in bare mode.
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. 

pytorch_model.bin:   2%|2         | 10.5M/501M [00:00<?, ?B/s]

Some weights of the model checkpoint at cardiffnlp/twitter-roberta-base-sentiment-latest were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
2025-11-09 15:25:40.205 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-09 15:25:40.206 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-09 15:25:40.208 Thread 'M

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/501M [00:00<?, ?B/s]